In [1]:
import dwave
import numpy as np
import openml
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import plotly.express as px
from plotly.subplots import make_subplots
import dimod
from dwave.system import LeapHybridCQMSampler

# dataset = openml.datasets.get_dataset(312)

clf = RandomForestClassifier()

In [2]:
df = pd.read_csv("Dataset-stock-HON.csv")

In [3]:
df.head()

,Date,Close,High,Low,SMA5,SMA10,SMA20,SMA30,RSI,%K,...,Aroon Up 10,Aroon Down 10,Aroon Up 15,Aroon Down 15,Aroon Up 20,Aroon Down 20,Aroon Up 30,Aroon Down 30,Price Change,Class
0,2021-02-09,202.020004,203.800003,201.080002,201.706000,200.179999,202.876499,205.330666,40.507518,56.334874,...,20.0,80.0,100.000000,53.333333,100.0,40.0,90.000000,26.666667,-0.139999,1
1,2021-02-10,202.160004,203.350006,201.179993,202.310001,200.457999,202.540499,205.071333,44.744960,70.462949,...,30.0,90.0,100.000000,60.000000,95.0,45.0,93.333333,30.000000,-0.019989,1
2,2021-02-11,202.179993,203.100006,200.880005,202.628000,200.391998,202.298499,204.839666,49.866653,70.648032,...,40.0,100.0,26.666667,66.666667,100.0,50.0,96.666667,33.333333,-1.390015,1
3,2021-02-12,203.570007,203.789993,201.649994,202.758002,201.212000,202.046499,204.593666,53.118147,83.518534,...,50.0,100.0,33.333333,73.333333,90.0,55.0,100.000000,36.666667,0.130005,0
4,2021-02-16,203.440002,205.570007,201.880005,202.674002,202.000000,201.919999,204.285000,54.477062,80.671470,...,60.0,90.0,40.000000,80.000000,95.0,60.0,90.000000,40.000000,1.540009,0


In [4]:
df.drop('Price Change',axis=1,inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473 entries, 0 to 472
Data columns (total 28 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           473 non-null    object 
 1   Close          473 non-null    float64
 2   High           473 non-null    float64
 3   Low            473 non-null    float64
 4   SMA5           473 non-null    float64
 5   SMA10          473 non-null    float64
 6   SMA20          473 non-null    float64
 7   SMA30          473 non-null    float64
 8   RSI            473 non-null    float64
 9   %K             473 non-null    float64
 10  %D             473 non-null    float64
 11  ATR            473 non-null    float64
 12  ADX            473 non-null    float64
 13  EMA5           473 non-null    float64
 14  EMA10          473 non-null    float64
 15  EMA20          473 non-null    float64
 16  EMA30          473 non-null    float64
 17  Aroon Up 5     473 non-null    float64
 18  Aroon Down

In [6]:
# from sklearn.datasets import load_breast_cancer
from dwave.plugins.sklearn import SelectFromQuadraticModel

X = df.drop(['Class','Date'], axis=1)
y = df['Class']


X.head()

,Close,High,Low,SMA5,SMA10,SMA20,SMA30,RSI,%K,%D,...,Aroon Up 5,Aroon Down 5,Aroon Up 10,Aroon Down 10,Aroon Up 15,Aroon Down 15,Aroon Up 20,Aroon Down 20,Aroon Up 30,Aroon Down 30
0,202.020004,203.800003,201.080002,201.706000,200.179999,202.876499,205.330666,40.507518,56.334874,60.390635,...,40.0,100.0,20.0,80.0,100.000000,53.333333,100.0,40.0,90.000000,26.666667
1,202.160004,203.350006,201.179993,202.310001,200.457999,202.540499,205.071333,44.744960,70.462949,64.244224,...,60.0,100.0,30.0,90.0,100.000000,60.000000,95.0,45.0,93.333333,30.000000
2,202.179993,203.100006,200.880005,202.628000,200.391998,202.298499,204.839666,49.866653,70.648032,65.815285,...,80.0,60.0,40.0,100.0,26.666667,66.666667,100.0,50.0,96.666667,33.333333
3,203.570007,203.789993,201.649994,202.758002,201.212000,202.046499,204.593666,53.118147,83.518534,74.876505,...,100.0,80.0,50.0,100.0,33.333333,73.333333,90.0,55.0,100.000000,36.666667
4,203.440002,205.570007,201.880005,202.674002,202.000000,201.919999,204.285000,54.477062,80.671470,78.279345,...,40.0,100.0,60.0,90.0,40.000000,80.000000,95.0,60.0,90.000000,40.000000


In [7]:
X_new = SelectFromQuadraticModel(num_features=5).fit_transform(X, y)


/home/naman/Documents/RASSE/ocean/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but SelectFromQuadraticModel was fitted without feature names
  warnings.warn(


In [8]:
print(X_new.shape)

X_new

(473, 5)


array([[  3.81428419,  28.44460018,  40.        ,  80.        ,
         90.        ],
       [  3.75642831,  26.82693867,  60.        ,  90.        ,
         93.33333333],
       [  3.64071437,  24.04577067,  80.        , 100.        ,
         96.66666667],
       ...,
       [  4.25999996,  15.07932902, 100.        ,  40.        ,
         60.        ],
       [  4.22714342,  13.10522721,  20.        ,  50.        ,
         63.33333333],
       [  4.29642923,  12.65183248,  40.        ,  60.        ,
         66.66666667]])

In [9]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score

In [10]:
def train_test_models(X, y):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Create a list to store the results
    results = []
    
    # Models
    models = {
        'SVM': SVC(),
        'Decision Tree': DecisionTreeClassifier(),
        'Random Forest': RandomForestClassifier(),
        'KNN': KNeighborsClassifier(),
        'Logistic Regression': LogisticRegression(max_iter=1500),
        'Naive Bayes': GaussianNB(),
        'Gradient Boosting': GradientBoostingClassifier(),
        'XGBoost': xgb.XGBClassifier()
    }
    
    for model_name, model in models.items():
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Calculate accuracy and F1 score
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # Store the results as a dictionary
        results.append({'Model': model_name, 'Accuracy': accuracy, 'F1 Score': f1})
    
    # Create a DataFrame from the results list
    results_df = pd.DataFrame(results)
    
    return results_df

In [11]:

# Train and test the models
results = train_test_models(X_new, y)

# Print the results
results.head(10)

,Model,Accuracy,F1 Score
0,SVM,0.526316,0.628099
1,Decision Tree,0.515789,0.566038
2,Random Forest,0.515789,0.574074
3,KNN,0.621053,0.689655
4,Logistic Regression,0.536842,0.607143
5,Naive Bayes,0.515789,0.581818
6,Gradient Boosting,0.494737,0.547170
7,XGBoost,0.505263,0.560748


In [12]:
X_new3 = SelectFromQuadraticModel(num_features=3).fit_transform(X, y)


/home/naman/Documents/RASSE/ocean/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but SelectFromQuadraticModel was fitted without feature names
  warnings.warn(


In [13]:
# Train and test the models
results = train_test_models(X_new3, y)

# Print the results
results.head(10)

,Model,Accuracy,F1 Score
0,SVM,0.505263,0.576577
1,Decision Tree,0.526316,0.587156
2,Random Forest,0.473684,0.509804
3,KNN,0.452632,0.535714
4,Logistic Regression,0.515789,0.557692
5,Naive Bayes,0.473684,0.519231
6,Gradient Boosting,0.473684,0.537037
7,XGBoost,0.421053,0.466019


In [14]:
X_new8 = SelectFromQuadraticModel(num_features=8).fit_transform(X, y)

/home/naman/Documents/RASSE/ocean/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but SelectFromQuadraticModel was fitted without feature names
  warnings.warn(


In [15]:
# Train and test the models
results = train_test_models(X_new8, y)

# Print the results
results.head(10)

,Model,Accuracy,F1 Score
0,SVM,0.463158,0.540541
1,Decision Tree,0.473684,0.528302
2,Random Forest,0.526316,0.579439
3,KNN,0.484211,0.533333
4,Logistic Regression,0.515789,0.557692
5,Naive Bayes,0.463158,0.532110
6,Gradient Boosting,0.505263,0.525253
7,XGBoost,0.526316,0.579439


In [16]:
num_cols = X_new.shape[1]
col_names = [f'col{i+1}' for i in range(num_cols)]
qa5 = pd.DataFrame(data=X_new, columns=col_names)

# Save the DataFrame to a CSV file
qa5.to_csv('QA-5-features-HON.csv', index=False)

In [17]:
num_cols = X_new3.shape[1]
col_names = [f'col{i+1}' for i in range(num_cols)]
qa3 = pd.DataFrame(data=X_new3, columns=col_names)

# Save the DataFrame to a CSV file
qa3.to_csv('QA-3-features-HON.csv', index=False)

In [18]:
num_cols = X_new8.shape[1]
col_names = [f'col{i+1}' for i in range(num_cols)]
qa8 = pd.DataFrame(data=X_new8, columns=col_names)

# Save the DataFrame to a CSV file
qa8.to_csv('QA-8-features-HON.csv', index=False)